# PaloAlto API

In [ ]:
import requests
import xmltodict
import urllib3
import time
from urllib.parse import quote
import pandas as pd
import geoip2.database
import json

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
pd.set_option('display.max_columns', None)

## API key request

In [ ]:
fw_url = "https://10.170.1.1/api/"

In [ ]:
url = '{0}?type=keygen&user=dc\user1&password=pass1'.format(fw_url)
r = requests.get(url, verify=False)

In [ ]:
dict_data = xmltodict.parse(r.content)
api_key = dict_data["response"]["result"]["key"]
api_key

## Get traffic logs

In [ ]:
#query = "(receive_time geq '2021/05/03 00:49:00')"
#query = "( user.src eq 'dc\user1')"
log_type = "traffic"
query = "( receive_time leq '2021/05/10 02:15:07' ) and ( receive_time geq '2021/05/09 02:15:07' )"

#log_type = "auth"
#query = "( description contains 'dc\user1' )" log_type = "system" # Para ver logins, accesos a consolas, 

#query = "( eventid eq auth-success )" 
#log_type = "system"

In [ ]:
log_type

In [ ]:
quoted_query = quote(query)
quoted_query

In [ ]:
def delete_job(api_key, job_id):
    url_job_delete = "{0}?key={1}&type=log&action=finish&job-id={2}".format(fw_url, api_key, job_id)
    job_delete_response = requests.get(url_job_delete, verify=False)
    print(xmltodict.parse(job_delete_response.content)["response"]["msg"])

In [ ]:
def fw_request(key, q_type, log_type='', skip=0, query='', action='', job_id=''):
    if "finish" in action:
        url = "{0}?key={1}&type=log&action=finish&job-id={2}".format(fw_url, api_key, job_id)
    elif "get" in action:
        url = fw_url + "{0}?key={1}&type=log&action=get&job-id={2}".format(fw_url,api_key,job_id_int)
    elif query:
        quoted_query = quote(query)
        url = fw_url + "{0}?key={1}&type=log&log-type={2}&nlogs=5000&skip={3}&query={4}".format(fw_url,api_key,log_type,skip,quoted_query)
    else:
        print("Invalid request")
        return False
    
    response = requests.get(url, verify=False)
    response_dict = xmltodict.parse(response.content)
    return response_dict
        

In [ ]:
%%time
# Log requests, download in groups of 5k events.
step = 0
last_query = False
from IPython.display import clear_output
while not last_query:
    clear_output(wait=True)
    skip = 5000 * step
    job_id_response = fw_request(api_key, "log", log_type, skip=skip, query=query)
    job_id_int = int(job_id_response["response"]["result"]["job"])
    print("Job ID: {0}".format(job_id_int))
    finished = False
    while not finished:
        data_query_response_dict = fw_request(api_key, "log", action='get', job_id=job_id_int)
        status = data_query_response_dict["response"]["result"]["job"]["status"]
        if "FIN" not in status:
            print("Status: {0}".format(status))
            #time.sleep(1)
        else:
            print("Status: {0}".format(status))
            fw_request(api_key, "log", action='finish', job_id=job_id_int)
            if int(data_query_response_dict["response"]["result"]["log"]["logs"]['@count']) > 0:
                #print(data_query_response_dict)
                entries = data_query_response_dict["response"]["result"]["log"]["logs"]['entry']
                if step == 0:
                    df = pd.DataFrame(entries)
                else:
                    df_t = pd.DataFrame(entries)
                    df = df.append(df_t, ignore_index=True)
                if int(data_query_response_dict["response"]["result"]["log"]["logs"]['@count']) < 5000:
                    print("Last query")
                    last_query = True
                step += 1
                finished = True
            else: 
                print("Query sin resultados")
                last_query = True
                break

In [ ]:
len(df.index)

In [ ]:
df

# System logs for authentication
The interesting data is in the column "opaque"  
Example: "authenticated for user 'dom\user01'.   auth profile 'AUTH_GlobalProtect', vsys 'vsys1', server profile 'Pool de Servidores AD', server address '10.1.10.23', From: 20.13.93.250."

### Extract data from column

In [ ]:
#Sytem logs ( eventid eq auth-success )
df["user.src"] = df["opaque"].str.extract("(?<=user ')(.*)(?=\'\.)", expand=True)
df["srv_address"] = df["opaque"].str.extract("(?<=ess ')(.*)(?=\',)", expand=True)
df["from"] = df["opaque"].str.extract("(?<=rom: )((\d){1,3}.(\d){1,3}.(\d){1,3}.(\d){1,3})", expand=True)
df.iloc[1]

### Users with connections from more than one country

In [ ]:
df[["from", "user.src"]].groupby("user.src").agg({"from": "nunique"}).sort_values("from", ascending=False).head()

In [ ]:
df[df["user"].str.contains("user1", na=False)][["from", "user.src"]].groupby("from").agg({"user.src": "count"}).sort_values("user.src", ascending=False).head()

In [ ]:
df[df["user.src"].str.contains("user1", na=False) & ~(df["from"].str.startswith("189", na=False))][["from", "user.src"]].groupby("from").agg({"user.src": "count"}).sort_values("user.src").head()

## GeoIP

In [ ]:
def get_country(ip):
    try:
        x = reader.country(ip)
    except:
        return pd.np.nan
    try:
        return x.country.name if x else pd.np.nan
    except KeyError:
        return pd.np.nan

In [ ]:
%%time
with geoip2.database.Reader('.\\geoip\\GeoLite2-Country.mmdb') as reader:
    # get unique IPs
    unique_ips = df['from'].unique()
    # make series out of it
    unique_ips = pd.Series(unique_ips, index = unique_ips)
    df['country'] = df['from'].map(unique_ips.apply(get_country))
    

#### Countries by number of legit accesses

In [ ]:
df[["country","@logid"]].groupby("country").agg({"@logid": "count"}).sort_values("@logid", ascending=False)

### Users with accesses from more than one country

In [ ]:
df[["country","user"]].groupby("user").agg({"country": "nunique"}).sort_values("country", ascending=False)

In [ ]:
df2 = df[["country","user"]].groupby("user").agg({"country": "nunique"})
df2[df2['country'] > 1]

In [ ]:
df2["countries"] = df.groupby(['user']).agg({"country": "unique"})['country']

In [ ]:
df2[df2.country > 1]

### User query

In [ ]:
df[(df["user"] == "mx\\user1") & (df["country"] == "Czechia")].groupby("from").agg({"receive_time": "count"})

In [ ]:
df[(df["user"] == "mx\\user1") & (df["country"] == "Czechia")]

### Abuse IP DB

In [ ]:
def abuseipdb_req(ip):
    try:
        key = "api-key"
        url = 'https://api.abuseipdb.com/api/v2/check'
        querystring = {
            'ipAddress': ip,
            'maxAgeInDays': '90'
        }
        headers = {
            'Accept': 'application/json',
            'Key': key
        }
        response = requests.request(method='GET', url=url, headers=headers, params=querystring)
        decodedResponse = json.loads(response.text)
    except:
        return pd.np.nan
    try:
        return decodedResponse["data"]["abuseConfidenceScore"] if decodedResponse else pd.np.nan
    except KeyError:
        return pd.np.nan

In [ ]:
abuse_ips = df[(df.user.isin(df2[df2.country > 1].index)) & (df["country"] != "Mexico")]["from"].unique()
len(abuse_ips)

In [ ]:
%%time
unique_ips = pd.Series(abuse_ips, index = abuse_ips)
df['abuse_result'] = df['from'].map(unique_ips.apply(abuseipdb_req))

In [ ]:
df[["from", "abuse_result"]].sort_values(by=["abuse_result"], ascending=False)

In [ ]:
df3 = df[["from", "abuse_result"]].groupby(["from"]).agg({"abuse_result": "mean"}).sort_values(by=["abuse_result"], ascending=False)
df3 = df3.reset_index()
df3[df3["abuse_result"] > 0]

### Tor Nodes

In [ ]:
nodes = requests.request(method='GET', url="https://www.dan.me.uk/torlist/") #30 minutes limit

In [ ]:
nodes_list = nodes.text.split("\n")

In [ ]:
def is_tor_node(ip):
    if ip in nodes_list:
        return True
    else:
        return False

In [ ]:
df3[df3["abuse_result"] > 0]
df3['is_tor_node'] = df3['from'].apply(is_tor_node)

In [ ]:
df3[df3["is_tor_node"]]